In [1]:
from bs4 import BeautifulSoup
import bs4
from selenium import webdriver
from selenium.webdriver import Safari
import requests
import pandas as pd

In [40]:
def getJobInfo(url, driver):
    
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    try:
        jobTitle = soup.title.text.split(' | ')[0]
    except AttributeError:
        return None
    jobID = int(soup.find('code', {"id":'decoratedJobPostingId'}).string[1:-1])
    
    company = jobTitle.split(' hiring ')[0].replace('"', '')
    potision = jobTitle.split(' hiring ')[1].split(' in ')[0].replace('"', '')
    location = jobTitle.split(' hiring ')[1].split(' in ')[1].replace('"', '')
    try:
        location = location.split(', ')[2]
    except:
        try:
            location = location.split(', ')[1]
        except:
            pass

    # TODO: need to find a better way to get requirements
    keywords = ["require", "qualifi", "skill", "have", "looking for", "must be", "ideal", "succ"]
    description = soup.find('div', {"class":'description__text description__text--rich'})
    for br in description.find_all("br"):
        br.extract()
    lists = description.find_all("ul")

    requirements = ""
    for ulist in lists:
        try:
            if any(keyword in ulist.previous_sibling.text.lower() for keyword in keywords):
                requirements = [requirement.text for requirement in ulist.find_all('li')]
                requirements = " ".join(requirements).replace('\n', ' ').replace(';', ',')
            elif any(keyword in ulist.previous_sibling.previous_sibling.find('strong').text.lower() for keyword in keywords):
                requirements = [requirement.text for requirement in ulist.find_all('li')]
                requirements = " ".join(requirements).replace('\n', ' ')
            elif any(keyword in ulist.previous_sibling.previous_sibling.find('p').text.lower() for keyword in keywords):
                requirements = [requirement.text for requirement in ulist.find_all('li')]
                requirements = " ".join(requirements).replace('\n', ' ')
        except AttributeError:
            pass
    
    
    row = [jobID, company, potision, location, requirements, url]

    return row

In [42]:
def saveJobsCsvFromUrls(txtfile='urls.txt', urlList=None, csvfile='jobs.csv'):
    
    if urlList == None:
        f = open(txtfile, 'r')
        urlList = f.readlines()
        urlList = [x.strip() for x in urlList]
        f.close()

    try:
        oldJobs = pd.read_csv(csvfile, sep=';')
    except:
        oldJobs = pd.DataFrame(columns=['JobID', 'Company', 'Position', 'Location', 'Requirements', 'JobUrl', 'Interview Status', 'Offer Status'])
    
    driver = webdriver.Safari()
    rows = []
    failedJobs = 0
    for url in urlList:
        jobid = int(url.split('/')[-2])
    
        if jobid in list(oldJobs['JobID']):
            continue
        # sometimes the web driver gets stuck when retrieving the page. if that's the case, we eill retry 5 times
        counter = 0
        while counter < 5:
            row = getJobInfo(url, driver)
            if row is not None:
                print("parsed job: " + row[1] + " " + row[2])
                rows.append(row)
                break
            counter += 1
        
        if counter == 5:
            print("failed to parse job: " + url)
            failedJobs += 1
            
    driver.quit()

    jobs = pd.DataFrame(rows, columns=['JobID', 'Company', 'Position', 'Location', 'Requirements', 'JobUrl'])

    jobs['Interview Status'] = 'Pending'
    jobs['Offer Status'] = 'Pending'


#add non-duplicate rows to oldJobs
    newJobs = jobs[~jobs['JobID'].isin(oldJobs['JobID'])]
    oldJobs = pd.concat([oldJobs, newJobs])
    oldJobs.to_csv(csvfile, index=False, sep=';')
    return failedJobs

saveJobsCsvFromUrls(csvfile='jobs4.csv')





parsed job: Orbis Junior Data Engineer
parsed job: European Recruitment Graduate Software Developer - STEM / Physics / Mathematics
parsed job: SoundHound AI UPCOMING: Software Engineer, Conversational AI
parsed job: Microsoft Data Scientist


0